In [ ]:
'''Пример парсера Кинопоиска. Автор (девушка, кстати) парсит выдачу с оценками фильмов на своем аккаунте
, а у меня ни аккаунта,ни тем более оценок там - нет.
Поэтому, только для разбора подхода парсинга'''

In [ ]:
'''Часть 1 - Загрузка данных'''

In [ ]:
import requests
# вводные
s = requests.Session() # использует сессию, как и Мурренган
s.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
    })
user_id = 12345
page = 1

def load_user_data(user_id, page, s):
    url = 'http://www.kinopoisk.ru/user/%d/votes/list/ord/date/page/%d/#list' % (user_id, page)
    # %d - это первый парметр урла. а следующийй %d - это второй. После % в скобках идут сами парметры
    request = session.get(url)
    return request.text #на выходе мы получаем текст страницы

def contain_movies_data(text):
    soup = BeautifulSoup(text) #передаем текст в суп
    film_list = soup.find('div', {'class': 'profileFilmsList'}) #ищет тег с оценкой
    return film_list is not None # интересный ход, делает ретурн только если есть что-то? Есл ретурна нет, то закончится перебор в цикле while

#Отсюда стартуем запуск. Ктсати она (автор) при парсинге страниц использует не for цикл, а while
#она не знает сколько страниц, поэтому будет перебирать пока есть страница с оценками.
#т.е пока будет находится тег {'class': 'profileFilmsList'}

while True:
    data = load_user_data(user_id, page, s)#запускаем первую функцию и передаем ее вывод в дата
    if contain_movies_data(data): #вот это как раз и есть условие для перебора.Пока будет ретурн от второй функции
        with open('./page_%d.html' % (page), 'w') as output_file:
            #сохраняем в домашнюю (./) папку файл с именем page_номер страницы на запись
            output_file.write(data.encode('cp1251'))
            page += 1
    else:
            break# завершить цикл, если стрниц больше нет.

In [ ]:
'''Часть 2 - Парсинг страниц'''

In [ ]:
'''Автор не указала точку входа'''

def read_file(filename):
    with open(filename) as input_file:
        text = input_file.read()
    return text

def parse_user_datafile_bs(filename):
    results = []
    text = read_file(filename)

    soup = BeautifulSoup(text)
    film_list = soup.find('div', {'class': 'profileFilmsList'}) #находим первый тего
    items = film_list.find_all('div', {'class': ['item', 'item even']}) # в нем находим все теги item even
    for item in items:
        # getting movie_id
#         movie_link = item.find('div', {'class': 'nameRus'}).find('a').get('href') #ссылка на фильм
        movie_desc = item.find('div', {'class': 'nameRus'}).find('a').text  #название фильма
        movie_id = re.findall('\d+', movie_link)[0] #.re - это вроде как модуль регекспов.Здесь команда найти все /d+ и первый элмент. Без страницы не понять.

        # getting english name
        name_eng = item.find('div', {'class': 'nameEng'}).text

        #getting watch time
        watch_datetime = item.find('div', {'class': 'date'}).text
        date_watched, time_watched = re.match('(\d{2}\.\d{2}\.\d{4}), (\d{2}:\d{2})', watch_datetime).groups() #что за метод .groups?

        # getting user rating
        user_rating = item.find('div', {'class': 'vote'}).text
        if user_rating:
            user_rating = int(user_rating)

        results.append({
                'movie_id': movie_id,
                'name_eng': name_eng,
                'date_watched': date_watched,
                'time_watched': time_watched,
                'user_rating': user_rating,
                'movie_desc': movie_desc
            }) # сложила все в словарик
    return results

In [ ]:
'''ЧТо узнал нового.
Надо глянуть на метод .re и .groups()
Это ссылка на саму статью, мало ли https://habr.com/ru/post/280238/
В самой статье есть примечания. что делать, если на сайте аутентификация (как ее пройти из питон-кода),
а также , как обрабатывать разные контролсы. И ссылка на курс Udacity
'''